# Generic physical transmission line example

In [ ]:
%load_ext autoreload
%autoreload 2
import skrf as rf
import skrf.mathFunctions as mf
import numpy as np
from numpy import real, log, log10, sum, absolute, pi, sqrt
import matplotlib.pyplot as plt
from scipy.optimize import minimize, differential_evolution

rf.stylely()

## Measurement of two CPWG lines with different lenghts

The measurement where performed the 21th March 2017 on a Anritsu MS46524B 20GHz Vector Network Analyser. The setup is a linear frequency sweep from 1MHz to 10GHz with 10'000 points. Output power is 0dBm, IF bandwidth is 1kHz and neither averaging nor smoothing are used.

CPWGxxx is a L long, W wide, with a G wide gap to top ground, T thick copper coplanar waveguide on ground on a H height substrate with top and bottom ground plane. A closely spaced via wall is placed on both side of the line and the top and bottom ground planes are connected by many vias.

| Name | L (mm) | W (mm) | G (mm) | H (mm) | T (um) | Substrate |
| :--- | ---: | ---: | ---: | ---: | ---: | :--- |
| MSL100 | 100 | 1.70 | 0.50 | 1.55 | 50 | FR-4 |
| MSL200 | 200 | 1.70 | 0.50 | 1.55 | 50 | FR-4 |

The milling of the artwork is performed mechanically with a lateral wall of 45°.

The relative permittivity of the dielectric was assumed to be approximatively 4.5 for design purpose.

![MSL100 and MSL200 iillustaration, both are microstripline, MSL200 is twice the length of MSL100](MSL_CPWG_100_200.jpg "MSL100 and MSL200")

In [ ]:
# Load raw measurements
TL100 = rf.Network('CPWG100.s2p')
TL200 = rf.Network('CPWG200.s2p')

plt.figure()
plt.title('Raw measurement')
TL100.plot_s_db()
TL200.plot_s_db()
plt.show()

## Dielectric effective relative permittivity extraction by multiline method

In [ ]:
#Make the missing reflect measurement
#Reflect only affects sign of the corrected
reflect = TL100.copy()
reflect.s[:,0,0] = 1
reflect.s[:,1,1] = 1
reflect.s[:,1,0] = 0
reflect.s[:,0,1] = 0

# Perform NISTMultilineTRL algorithm
cal = rf.NISTMultilineTRL([TL100, reflect, TL200], [1], [100e-3, 200e-3], er_est=3.0, refl_offset=[0])

plt.figure()
plt.title('Corrected lines')
cal.apply_cal(TL100).plot_s_db()
cal.apply_cal(TL200).plot_s_db()
plt.show()

In [ ]:
freq  = TL100.frequency
f     = TL100.frequency.f
f_ghz = TL100.frequency.f/1e9
L     = 0.1
A     = 0.0
f_A   = 1e9
Er0   = 2.0
tand0 = 0.001
f_epr_tand = 1e9
x0 = [Er0, tand0]

ep_r_mea = cal.er_eff.real
A_mea    = 20/log(10)*cal.gamma.real

def model(x, freq, ep_r_mea, A_mea, f_epr_tand, L):
    ep_r = x[0]
    tand = x[1]
    m = rf.media.PhysicalLine(frequency=freq, z0=50, ep_r=ep_r, mu_r=1, tand=tand,
                              f_low=1e3, f_high=1e18, f_epr_tand=f_epr_tand, diel='djordjevicsvensson')
    ep_r_mod = m.ep_r_f.real
    A_mod = m.alpha * log(10)/20
    return sum((ep_r_mod - ep_r_mea)**2)  + 0.001*sum((20/log(10)*A_mod - A_mea)**2)

res = minimize(model, x0, args=(TL100.frequency, ep_r_mea, A_mea, f_epr_tand, L),
               bounds=[(2, 4), (0.001, 0.013)])
ep_r = res.x[0]
tand = res.x[1]

print('epr={:.3f}, tand={:.4f} at {:.1f} GHz.'.format(ep_r, tand, f_epr_tand * 1e-9))

m = rf.media.PhysicalLine(frequency=freq, z0=50, ep_r=ep_r, mu_r=1, tand=tand,
                              f_low=1e3, f_high=1e18, f_epr_tand=f_epr_tand, diel='djordjevicsvensson')

plt.figure()
plt.suptitle('Effective relative permittivity and attenuation')
plt.subplot(2,1,1)
plt.ylabel('$\epsilon_{r,eff}$')
plt.plot(f_ghz, ep_r_mea, label='measured')
plt.plot(f_ghz, m.ep_r_f.real, label='model')
plt.legend()

plt.subplot(2,1,2)
plt.xlabel('Frequency [GHz]')
plt.ylabel('A (dB/m)')
plt.plot(f_ghz, A_mea, label='measured')
plt.plot(f_ghz, 20/log(10)*m.alpha, label='model')
plt.legend()
plt.show()


## Connectors effects estimation

In [ ]:
Z0 = 51.5 # Guessed for time being

half = m.line(L/2, 'm', embed=True, z0=Z0)

coefs = cal.coefs
r = mf.sqrt_phase_unwrap(coefs['forward reflection tracking'])
s1 = np.array([[coefs['forward directivity'],r],
        [r, coefs['forward source match']]]).transpose()

conn = TL100.copy()
conn.name = 'Connector'
conn.s = s1
conn = conn ** half.inv

# delay estimation
phi_conn = np.unwrap(np.angle(conn.s[:2000,1,0]))
z = np.polyfit(f[:2000], phi_conn, 1)
p = np.poly1d(z)
delay = -z[0]/(2*np.pi)/2
print('Connector delay: {:.0f} ps'.format(delay * 1e12))

# connector model
Z1 = 58 # Guessed for time being
m2 = rf.media.PhysicalLine(m.frequency, z0=50, ep_r=1, mu_r=1, tand=0.02, A=0, f_A=1e9,
                              f_low=1e3, f_high=1e12, f_epr_tand=1e9, diel='djordjevicsvensson')
left = m2.line(delay * 1e9, 'ns', embed=True, z0=Z1)
right = left.flipped()
check = left ** right

plt.figure()
plt.suptitle('Connector effects')
plt.subplot(2,1,1)
plt.plot(f_ghz[:2000], phi_conn, label='measured')
plt.plot(f_ghz, np.unwrap(np.angle(check.s[:,1,0])), label='model')
plt.ylabel('phase (rad)')
plt.legend()

plt.subplot(2,1,2)
conn.plot_s_db(1, 0, label='Measured')
plt.plot(f_ghz, 20*np.log10(np.absolute(check.s[:,1,0])), label='model')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Insertion Loss (dB)')
plt.legend()
plt.show()

## Final check

In [ ]:
DUT = m.line(L, 'm', embed=True, z0=51.5)
DUT.name = 'model'

Check = left ** DUT ** right
Check.name = 'model with connectors'

plt.figure()
TL100.plot_s_db()
Check.plot_s_db(1,0, color='k')
Check.plot_s_db(0,0, color='k')
plt.show()

mod = left ** DUT ** right

TL100_dc = TL100.extrapolate_to_dc(kind='linear')
mod_dc = mod.extrapolate_to_dc(kind='linear')

plt.figure()
plt.suptitle('Left-right and right-left TDR')
plt.subplot(2,1,1)
TL100_dc.s11.plot_s_time_step(pad=2000, window='hamming', label='Measured L-R')
mod_dc.s11.plot_s_time_step(pad=2000, window='hamming', label='Model L-R')
plt.xlim(-2e-9, 4e-9)

plt.subplot(2,1,2)
TL100_dc.s22.plot_s_time_step(pad=2000, window='hamming', label='Measured R-L')
mod_dc.s22.plot_s_time_step(pad=2000, window='hamming', label='Model R-L')
plt.xlim(-2e-9, 4e-9)